In [33]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plot

In [34]:
import xgboost as xgb

In [35]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = [
'https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive',
]

json_file_name = 'omega-tenure-341805-e2f94364a249.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)


spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1Ws4AoSLvvlJs8ZOrN7lpF6Y1PWwrZUVssF71Awatmuw/edit#gid=448337019'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 일반 지표
samsung = doc.worksheet('df')

In [36]:
#삼성전자와 친구들
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1N5Gv5H5CgYYPzgT8Jrl4KeIdmrWtXAqqcBsVQ-tcS-M/edit#gid=0'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
samsung_fund = doc.worksheet('시트1')

In [37]:
samsung = pd.DataFrame(samsung.get_all_records())
samsung_fund = pd.DataFrame(samsung_fund.get_all_records())

In [38]:
samsung.head(3)

,시가,상한가,하한가,종가,거래량,등락률,Date,감정
0,55500,56000,55000,55200,12993228,-0.010753,2020-01-02,0.785973
1,56000,56600,54900,55500,15422255,0.005435,2020-01-03,2.880952
2,54900,55600,54600,55500,10278951,0.000000,2020-01-06,1.026229


In [39]:
samsung_fund.head(3)

,삼성 PBR,LG전자 PBR,SK하이닉스 PBR,삼성 PER,LG전자 PER,SK하이닉스 PER,삼성 ROE,Date
0,1.56,0.90,1.47,8.54,10.32,4.26,0.182670,2020-01-02
1,1.57,0.90,1.47,8.59,10.33,4.25,0.182771,2020-01-03
2,1.57,0.89,1.47,8.59,10.16,4.24,0.182771,2020-01-06


In [40]:
samsung_fund.tail(3)

,삼성 PBR,LG전자 PBR,SK하이닉스 PBR,삼성 PER,LG전자 PER,SK하이닉스 PER,삼성 ROE,Date
592,1.52,1.07,1.19,11.49,17.90,7.72,0.132289,2022-05-25
593,1.51,1.07,1.14,11.41,17.82,7.36,0.132340,2022-05-26
594,1.52,1.08,1.17,11.51,18.08,7.58,0.132059,2022-05-27


In [41]:
samsung.tail(3)

,시가,상한가,하한가,종가,거래량,등락률,Date,감정
592,66700,67100,65900,66400,15150490,-0.001504,2022-05-25,1.527778
593,66300,67200,65500,65900,15970890,-0.007530,2022-05-26,4.349578
594,66700,66900,66200,66500,11346700,0.009105,2022-05-27,3.015542


In [42]:
data = pd.merge(samsung, samsung_fund, left_on='Date', right_on='Date', how='inner')

In [43]:
data.head(1)

,시가,상한가,하한가,종가,거래량,등락률,Date,감정,삼성 PBR,LG전자 PBR,SK하이닉스 PBR,삼성 PER,LG전자 PER,SK하이닉스 PER,삼성 ROE
0,55500,56000,55000,55200,12993228,-0.010753,2020-01-02,0.785973,1.56,0.9,1.47,8.54,10.32,4.26,0.18267


In [44]:
data.tail(1)

,시가,상한가,하한가,종가,거래량,등락률,Date,감정,삼성 PBR,LG전자 PBR,SK하이닉스 PBR,삼성 PER,LG전자 PER,SK하이닉스 PER,삼성 ROE
594,66700,66900,66200,66500,11346700,0.009105,2022-05-27,3.015542,1.52,1.08,1.17,11.51,18.08,7.58,0.132059


In [45]:
data.columns

Index(['시가', '상한가', '하한가', '종가', '거래량', '등락률', 'Date', '감정', '삼성 PBR',
       'LG전자 PBR', 'SK하이닉스 PBR', '삼성 PER', 'LG전자 PER', 'SK하이닉스 PER', '삼성 ROE'],
      dtype='object')

In [46]:
data = data[['Date', '시가', '상한가', '하한가', '종가', '거래량', '등락률', '감정', '삼성 PBR',
       'LG전자 PBR', 'SK하이닉스 PBR', '삼성 PER', 'LG전자 PER', 'SK하이닉스 PER']]

In [47]:
data.head(3)

,Date,시가,상한가,하한가,종가,거래량,등락률,감정,삼성 PBR,LG전자 PBR,SK하이닉스 PBR,삼성 PER,LG전자 PER,SK하이닉스 PER
0,2020-01-02,55500,56000,55000,55200,12993228,-0.010753,0.785973,1.56,0.90,1.47,8.54,10.32,4.26
1,2020-01-03,56000,56600,54900,55500,15422255,0.005435,2.880952,1.57,0.90,1.47,8.59,10.33,4.25
2,2020-01-06,54900,55600,54600,55500,10278951,0.000000,1.026229,1.57,0.89,1.47,8.59,10.16,4.24


In [48]:
data.tail(3)

,Date,시가,상한가,하한가,종가,거래량,등락률,감정,삼성 PBR,LG전자 PBR,SK하이닉스 PBR,삼성 PER,LG전자 PER,SK하이닉스 PER
592,2022-05-25,66700,67100,65900,66400,15150490,-0.001504,1.527778,1.52,1.07,1.19,11.49,17.90,7.72
593,2022-05-26,66300,67200,65500,65900,15970890,-0.007530,4.349578,1.51,1.07,1.14,11.41,17.82,7.36
594,2022-05-27,66700,66900,66200,66500,11346700,0.009105,3.015542,1.52,1.08,1.17,11.51,18.08,7.58


In [49]:
data['종가']

0      55200
1      55500
2      55500
3      55800
4      56800
       ...  
590    67900
591    66500
592    66400
593    65900
594    66500
Name: 종가, Length: 595, dtype: int64

In [50]:
y = data['하한가'].shift(-1)

In [51]:
y = data['종가'].shift(-1)

In [52]:
y = y[:-1]

In [53]:
x = data.drop(data.index[-1:])

In [54]:
x = x.drop(['Date'], axis=1)

In [55]:
y

0      55500.0
1      55500.0
2      55800.0
3      56800.0
4      58600.0
        ...   
589    67900.0
590    66500.0
591    66400.0
592    65900.0
593    66500.0
Name: 종가, Length: 594, dtype: float64

In [56]:
x

,시가,상한가,하한가,종가,거래량,등락률,감정,삼성 PBR,LG전자 PBR,SK하이닉스 PBR,삼성 PER,LG전자 PER,SK하이닉스 PER
0,55500,56000,55000,55200,12993228,-0.010753,0.785973,1.56,0.90,1.47,8.54,10.32,4.26
1,56000,56600,54900,55500,15422255,0.005435,2.880952,1.57,0.90,1.47,8.59,10.33,4.25
2,54900,55600,54600,55500,10278951,0.000000,1.026229,1.57,0.89,1.47,8.59,10.16,4.24
3,55700,56400,55600,55800,10009778,0.005405,2.308939,1.58,0.89,1.46,8.64,10.22,4.22
4,56200,57400,55900,56800,23501171,0.017921,5.824271,1.61,0.86,1.51,8.79,9.81,4.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,67800,68400,67700,68000,12109671,0.007407,1.047101,1.56,1.09,1.24,11.77,18.25,8.04
590,68800,68800,67600,67900,13684088,-0.001471,2.329107,1.56,1.10,1.25,11.75,18.34,8.08
591,67500,67700,66500,66500,15482576,-0.020619,2.996825,1.52,1.07,1.20,11.51,17.90,7.76
592,66700,67100,65900,66400,15150490,-0.001504,1.527778,1.52,1.07,1.19,11.49,17.90,7.72


In [57]:
x = np.array(x)
y = np.array(y)

In [58]:
x.shape, y.shape

((594, 13), (594,))

In [59]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 970731)

In [60]:
x_train = x[:-1]
y_train = y[:-1]
x_test = x[-1]
y_test = y[-1]

In [61]:
x[:-1]

array([[5.550e+04, 5.600e+04, 5.500e+04, ..., 8.540e+00, 1.032e+01,
        4.260e+00],
       [5.600e+04, 5.660e+04, 5.490e+04, ..., 8.590e+00, 1.033e+01,
        4.250e+00],
       [5.490e+04, 5.560e+04, 5.460e+04, ..., 8.590e+00, 1.016e+01,
        4.240e+00],
       ...,
       [6.880e+04, 6.880e+04, 6.760e+04, ..., 1.175e+01, 1.834e+01,
        8.080e+00],
       [6.750e+04, 6.770e+04, 6.650e+04, ..., 1.151e+01, 1.790e+01,
        7.760e+00],
       [6.670e+04, 6.710e+04, 6.590e+04, ..., 1.149e+01, 1.790e+01,
        7.720e+00]])

In [62]:
x[-1]

array([ 6.63000000e+04,  6.72000000e+04,  6.55000000e+04,  6.59000000e+04,
        1.59708900e+07, -7.53012048e-03,  4.34957831e+00,  1.50999999e+00,
        1.07000005e+00,  1.13999999e+00,  1.14100000e+01,  1.78200000e+01,
        7.36000000e+00])

In [63]:
model = xgb.XGBRegressor(seed=0, max_depth=6, eta=0.3, random_state=970731)

In [64]:
model.fit(x_train, y_train)

[18:44:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear',
             random_state=970731, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=0, silent=None, subsample=1, verbosity=1)

In [65]:
print(model.score(x_train, y_train))

0.999157204979842


In [66]:
#싱글 피쳐일 경우
x_test = x_test.reshape(1, -1)

In [67]:
y_pred = model.predict(x_test)

In [68]:
print(y_pred)
#75193
#73756

[64648.26]


In [69]:
today_value = data.drop(['Date'], axis=1)

In [70]:
today_value = data.drop(['Date'], axis=1)
today_value = np.array(today_value)

In [71]:
today_value = today_value[-1]

In [72]:
today_value = today_value.reshape(1, -1)

In [73]:
predict = model.predict(today_value)

In [74]:
predict

array([66297.03], dtype=float32)

In [75]:
daily_samsung_data = data
daily_samsung_data = daily_samsung_data.tail(1)

In [76]:
daily_samsung_data['예측종가'] = int(predict)

F:\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
int(daily_samsung_data['예측종가'].values)

66297

In [78]:
if daily_samsung_data['예측종가'].values > daily_samsung_data['종가'].values:
    daily_samsung_data['텍스트1'] = ("⇧")
if daily_samsung_data['예측종가'].values < daily_samsung_data['종가'].values:
    daily_samsung_data['텍스트1'] = ("⇩")

F:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [79]:
y = data['하한가'].shift(-1)
y = y[:-1]
x = data.drop(data.index[-1:])
x = x.drop(['Date'], axis=1)
x = np.array(x)
y = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 970731)
x_train = x[:-1]
y_train = y[:-1]
x_test = x[-1]
y_test = y[-1]
model = xgb.XGBRegressor(seed=0, max_depth=6, eta=0.3, random_state=970731)
model.fit(x_train, y_train)
today_value = data.drop(['Date'], axis=1)
today_value = data.drop(['Date'], axis=1)
today_value = np.array(today_value)
today_value = today_value[-1]
today_value = today_value.reshape(1, -1)
predict = model.predict(today_value)
daily_samsung_data['예측하한가'] = int(predict)

[18:44:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


F:\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [80]:
y = data['상한가'].shift(-1)
y = y[:-1]
x = data.drop(data.index[-1:])
x = x.drop(['Date'], axis=1)
x = np.array(x)
y = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 970731)
x_train = x[:-1]
y_train = y[:-1]
x_test = x[-1]
y_test = y[-1]
model = xgb.XGBRegressor(seed=0, max_depth=6, eta=0.3, random_state=970731)
model.fit(x_train, y_train)
today_value = data.drop(['Date'], axis=1)
today_value = data.drop(['Date'], axis=1)
today_value = np.array(today_value)
today_value = today_value[-1]
today_value = today_value.reshape(1, -1)
predict = model.predict(today_value)
daily_samsung_data['예측상한가'] = int(predict)

[18:44:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


F:\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [81]:
daily_samsung_data

,Date,시가,상한가,하한가,종가,거래량,등락률,감정,삼성 PBR,LG전자 PBR,SK하이닉스 PBR,삼성 PER,LG전자 PER,SK하이닉스 PER,예측종가,텍스트1,예측하한가,예측상한가
594,2022-05-27,66700,66900,66200,66500,11346700,0.009105,3.015542,1.52,1.08,1.17,11.51,18.08,7.58,66297,⇩,65841,67121


In [82]:
#데일리 삼성
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1hVL4cJBJkqHi80YXCp45Yfv79_iVVmVPrPHzGAv5OCQ/edit#gid=0'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
daily_samsung = doc.worksheet('시트1')

In [88]:
from datetime import datetime

In [89]:
day = datetime.today().strftime("%Y%m%d%H%M%S")

In [91]:
daily_samsung_data.to_csv(f"samsung_predict{day}.csv", encoding="UTF-8")

In [83]:
daily_samsung.update([daily_samsung_data.columns.values.tolist()] + daily_samsung_data.values.tolist())

{'spreadsheetId': '1hVL4cJBJkqHi80YXCp45Yfv79_iVVmVPrPHzGAv5OCQ',
 'updatedRange': "'시트1'!A1:R2",
 'updatedRows': 2,
 'updatedColumns': 18,
 'updatedCells': 36}